In [2]:
import pandas as pd
import mygene
client = mygene.get_client('gene')

In [ ]:
df = pd.read_csv("mainTable_all.csv", index_col=0)
df

In [ ]:
df_conversion = pd.read_csv("https://www.genenames.org/cgi-bin/download/custom?col=gd_app_sym&col=gd_pub_ensembl_id&status=Approved&status=Entry%20Withdrawn&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit", sep="\t", index_col=0)

In [ ]:
HVE_genes_ensg = []
for g in pd.read_csv("HDE_Breast.txt", header=None).values.ravel():
    if g in df_conversion.index:
        HVE_genes_ensg.append(df_conversion.at[g, 'Ensembl gene ID'])
    else:
        q = client.query(g, fields='symbol,ensembl')
        if len(q['hits'])>0:
            if  'ensembl' in q['hits'][0].keys():
                if len(q['hits'][0]['ensembl'])==0:
                    if 'gene' in q['hits'][0]['ensembl'].keys():
                        if 'ENSG' in q['hits'][0]['ensembl']['gene']:
                            HVE_genes_ensg.append(q['hits'][0]['ensembl']['gene'])
                        else:
                            print("NO ensg for %s"%g, q)
                    else:
                        print(g, q)
                else:
                    print(g, q)
        else:
            print(g, q)

In [ ]:
len(HVE_genes_ensg)

In [ ]:
len(pd.read_csv("HDE_Breast.txt", header=None))

In [ ]:
df[df.index.isin(HVE_genes_ensg)].to_csv("mainTable.csv", index=True, header=True)

In [ ]:
pd.DataFrame(data=HVE_genes_ensg).to_csv("genes.txt", index=False, header=False)

## Files

In [3]:
df_info = pd.read_csv("information.csv", header=1, index_col=0)
df_info = df_info.dropna(subset=['PAM50 mRNA'])

In [26]:
df_info = pd.read_csv("tcga_subset.csv", header=0, index_col=0)
df_info.set_index('pan.samplesID', inplace=True)
df_info.groupby(["Subtype_Selected","Subtype_mRNA"]).count()

,,cancer.type,Subtype_DNAmeth,Subtype_protein,Subtype_miRNA,Subtype_CNA,Subtype_Integrative,Subtype_other
Subtype_Selected,Subtype_mRNA,,,,,,,
BRCA.Basal,Basal,193,0,0,0,0,0,0
BRCA.Her2,Her2,82,0,0,0,0,0,0
BRCA.LumA,LumA,581,0,0,0,0,0,0
BRCA.LumB,LumB,219,0,0,0,0,0,0
BRCA.Normal,Normal,143,0,0,0,0,0,0


In [17]:
df_tcgabiolinks=pd.read_csv("TCGAbiolinks.csv", index_col=0)
df_tcgabiolinks.head()

,patient,Tumor.Type,Included_in_previous_marker_papers,vital_status,days_to_birth,days_to_death,days_to_last_followup,age_at_initial_pathologic_diagnosis,pathologic_stage,Tumor_Grade,...,tobacco_smoking_history,CNV Clusters,Mutation Clusters,DNA.Methylation Clusters,mRNA Clusters,miRNA Clusters,lncRNA Clusters,Protein Clusters,PARADIGM Clusters,Pan-Gyn Clusters
1,TCGA-3C-AAAU,BRCA,NO,Alive,-20211.0,NaN,4047.0,55,NaN,NaN,...,NaN,C6,C7,C1,C1,C3,NaN,NaN,C5,NaN
2,TCGA-3C-AALI,BRCA,NO,Alive,-18538.0,NaN,4005.0,50,Stage_II,NaN,...,NaN,C6,C9,C2,C2,C3,NaN,C2,C4,C4
3,TCGA-3C-AALJ,BRCA,NO,Alive,-22848.0,NaN,1474.0,62,Stage_II,NaN,...,NaN,C6,C4,C2,C2,C2,NaN,NaN,C4,NaN
4,TCGA-3C-AALK,BRCA,NO,Alive,-19074.0,NaN,1448.0,52,Stage_I,NaN,...,NaN,C1,C5,C2,C2,C2,NaN,C2,C6,C4
5,TCGA-4H-AAAK,BRCA,NO,Alive,-18371.0,NaN,348.0,50,Stage_III,NaN,...,NaN,C6,C4,C1,C2,C2,NaN,C2,C6,C1


In [125]:
df_files = pd.read_csv("files_all.dat", index_col=8, header=0)
df_files = df_files.transpose().append(df_info.transpose(), sort=False).transpose()

df_files.insert(0,'subtype', None)
df_files.insert(1,'vital_status', None)
df_files.insert(1,'pathologic_stage', None)

for sample in df_files.index:
    if sample in df_info.index:
        df_files.at[sample, 'subtype']=df_info.at[sample, 'Subtype_Selected']
        
        vs = df_tcgabiolinks[df_tcgabiolinks["patient"]==sample[:12]]['vital_status'].values
        if len(vs) ==1:
            df_files.at[sample, 'vital_status']= vs[0]
        else:
            df_files.at[sample, 'vital_status']= None
        
        d_t_d = df_tcgabiolinks[df_tcgabiolinks["patient"]==sample[:12]]['days_to_death'].values
        if len(d_t_d) == 1:
            df_files.at[sample, 'days_to_death']=round(d_t_d[0])
        else:
            df_files.at[sample, 'days_to_death']=None
            
        ps = df_tcgabiolinks[df_tcgabiolinks["patient"]==sample[:12]]['pathologic_stage'].values
        if len(ps) == 1:
            df_files.at[sample, 'pathologic_stage']=ps[0]
        else:
            df_files.at[sample, 'pathologic_stage']=None
        
    else:
        df_files.at[sample, 'subtype']=None
        df_files.at[sample, 'vital_status']=None
        
#bin days to death
bins = pd.np.linspace(df_files["days_to_death"].min(), df_files["days_to_death"].max(), num=10)
labels = range(len(bins)-1)
df_files["days_to_death_binned"] = pd.cut(df_files["days_to_death"], bins=bins, labels=labels)
df_files["days_to_death_binned"] = df_files["days_to_death_binned"].astype(int).values
df_files["days_to_death_binned"][df_files["days_to_death_binned"]<0] = -1
df_files.head()

/home/fvalle/anaconda3/envs/phd/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,subtype,pathologic_stage,vital_status,file_name,cases.0.diagnoses.0.prior_malignancy,cases.0.project.primary_site,cases.0.diagnoses.0.tumor_grade,cases.0.project.disease_type,cases.0.diagnoses.0.progression_or_recurrence,cases.0.diagnoses.0.tumor_stage,...,Subtype_mRNA,Subtype_DNAmeth,Subtype_protein,Subtype_miRNA,Subtype_CNA,Subtype_Integrative,Subtype_other,Subtype_Selected,days_to_death,days_to_death_binned
TCGA-A2-A0T5-01A-21R-A084-07,BRCA.LumA,Stage_II,Alive,22c16f46-950f-4389-bb16-b5cec4d8b94e.FPKM.txt.gz,no,Breast,not reported,Breast Invasive Carcinoma,not reported,stage iia,...,LumA,NaN,NaN,NaN,NaN,NaN,NaN,BRCA.LumA,NaN,-1
TCGA-A2-A0EM-01A-11R-A034-07,BRCA.LumA,Stage_I,Alive,90d19c5c-f80b-454b-b776-99d13f01617e.FPKM.txt.gz,no,Breast,not reported,Breast Invasive Carcinoma,not reported,stage ia,...,LumA,NaN,NaN,NaN,NaN,NaN,NaN,BRCA.LumA,NaN,-1
TCGA-BH-A0BJ-01A-11R-A056-07,BRCA.LumA,Stage_II,Alive,c443bbe4-b053-47fa-a7a9-97457e44e2a0.FPKM.txt.gz,no,Breast,not reported,Breast Invasive Carcinoma,not reported,stage iib,...,LumA,NaN,NaN,NaN,NaN,NaN,NaN,BRCA.LumA,NaN,-1
TCGA-A7-A0DC-01B-04R-A22O-07,None,None,None,7bcc02ad-a4c7-44e1-8654-10c4598f936d.FPKM.txt.gz,no,Breast,not reported,Breast Invasive Carcinoma,not reported,stage ia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
TCGA-D8-A146-01A-31R-A115-07,BRCA.LumA,Stage_II,Alive,26e8feed-d934-4dbc-8b04-d771b2a65915.FPKM.txt.gz,yes,Breast,not reported,Breast Invasive Carcinoma,not reported,stage iia,...,LumA,NaN,NaN,NaN,NaN,NaN,NaN,BRCA.LumA,NaN,-1


In [126]:
df_files.insert(0,'Subtype_Selected_L',df_files['Subtype_Selected'])
df_files.head()

,Subtype_Selected_L,subtype,pathologic_stage,vital_status,file_name,cases.0.diagnoses.0.prior_malignancy,cases.0.project.primary_site,cases.0.diagnoses.0.tumor_grade,cases.0.project.disease_type,cases.0.diagnoses.0.progression_or_recurrence,...,Subtype_mRNA,Subtype_DNAmeth,Subtype_protein,Subtype_miRNA,Subtype_CNA,Subtype_Integrative,Subtype_other,Subtype_Selected,days_to_death,days_to_death_binned
TCGA-A2-A0T5-01A-21R-A084-07,BRCA.LumA,BRCA.LumA,Stage_II,Alive,22c16f46-950f-4389-bb16-b5cec4d8b94e.FPKM.txt.gz,no,Breast,not reported,Breast Invasive Carcinoma,not reported,...,LumA,NaN,NaN,NaN,NaN,NaN,NaN,BRCA.LumA,NaN,-1
TCGA-A2-A0EM-01A-11R-A034-07,BRCA.LumA,BRCA.LumA,Stage_I,Alive,90d19c5c-f80b-454b-b776-99d13f01617e.FPKM.txt.gz,no,Breast,not reported,Breast Invasive Carcinoma,not reported,...,LumA,NaN,NaN,NaN,NaN,NaN,NaN,BRCA.LumA,NaN,-1
TCGA-BH-A0BJ-01A-11R-A056-07,BRCA.LumA,BRCA.LumA,Stage_II,Alive,c443bbe4-b053-47fa-a7a9-97457e44e2a0.FPKM.txt.gz,no,Breast,not reported,Breast Invasive Carcinoma,not reported,...,LumA,NaN,NaN,NaN,NaN,NaN,NaN,BRCA.LumA,NaN,-1
TCGA-A7-A0DC-01B-04R-A22O-07,NaN,None,None,None,7bcc02ad-a4c7-44e1-8654-10c4598f936d.FPKM.txt.gz,no,Breast,not reported,Breast Invasive Carcinoma,not reported,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
TCGA-D8-A146-01A-31R-A115-07,BRCA.LumA,BRCA.LumA,Stage_II,Alive,26e8feed-d934-4dbc-8b04-d771b2a65915.FPKM.txt.gz,yes,Breast,not reported,Breast Invasive Carcinoma,not reported,...,LumA,NaN,NaN,NaN,NaN,NaN,NaN,BRCA.LumA,NaN,-1


In [127]:
for sample in df_files[(df_files['Subtype_Selected_L']=='BRCA.LumA') | (df_files['Subtype_Selected_L']=='BRCA.LumB')].index:
    df_files.at[sample, 'Subtype_Selected_L']='BRCA.Lum'
df_files.head()

,Subtype_Selected_L,subtype,pathologic_stage,vital_status,file_name,cases.0.diagnoses.0.prior_malignancy,cases.0.project.primary_site,cases.0.diagnoses.0.tumor_grade,cases.0.project.disease_type,cases.0.diagnoses.0.progression_or_recurrence,...,Subtype_mRNA,Subtype_DNAmeth,Subtype_protein,Subtype_miRNA,Subtype_CNA,Subtype_Integrative,Subtype_other,Subtype_Selected,days_to_death,days_to_death_binned
TCGA-A2-A0T5-01A-21R-A084-07,BRCA.Lum,BRCA.LumA,Stage_II,Alive,22c16f46-950f-4389-bb16-b5cec4d8b94e.FPKM.txt.gz,no,Breast,not reported,Breast Invasive Carcinoma,not reported,...,LumA,NaN,NaN,NaN,NaN,NaN,NaN,BRCA.LumA,NaN,-1
TCGA-A2-A0EM-01A-11R-A034-07,BRCA.Lum,BRCA.LumA,Stage_I,Alive,90d19c5c-f80b-454b-b776-99d13f01617e.FPKM.txt.gz,no,Breast,not reported,Breast Invasive Carcinoma,not reported,...,LumA,NaN,NaN,NaN,NaN,NaN,NaN,BRCA.LumA,NaN,-1
TCGA-BH-A0BJ-01A-11R-A056-07,BRCA.Lum,BRCA.LumA,Stage_II,Alive,c443bbe4-b053-47fa-a7a9-97457e44e2a0.FPKM.txt.gz,no,Breast,not reported,Breast Invasive Carcinoma,not reported,...,LumA,NaN,NaN,NaN,NaN,NaN,NaN,BRCA.LumA,NaN,-1
TCGA-A7-A0DC-01B-04R-A22O-07,NaN,None,None,None,7bcc02ad-a4c7-44e1-8654-10c4598f936d.FPKM.txt.gz,no,Breast,not reported,Breast Invasive Carcinoma,not reported,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
TCGA-D8-A146-01A-31R-A115-07,BRCA.Lum,BRCA.LumA,Stage_II,Alive,26e8feed-d934-4dbc-8b04-d771b2a65915.FPKM.txt.gz,yes,Breast,not reported,Breast Invasive Carcinoma,not reported,...,LumA,NaN,NaN,NaN,NaN,NaN,NaN,BRCA.LumA,NaN,-1


In [135]:
df_files.set_index('file_name').dropna(axis=1, how='all').fillna("unknown").to_csv("files.dat", index=True, header=True)

In [ ]:
df[df_files.dropna(axis=0, subset=['RPPA'])['file_name'].values][df.index.isin(HVE_genes_ensg)].to_csv("mainTable.csv", index=True, header=True)

In [138]:
df_files.set_index('file_name').dropna(axis=1, how='all').fillna("unknown").loc["67943973-e57a-4285-bd6e-995053d23fe0.FPKM.txt.gz",:]

Subtype_Selected_L                                              BRCA.Basal
subtype                                                         BRCA.Basal
pathologic_stage                                                 Stage_III
vital_status                                                         Alive
cases.0.diagnoses.0.prior_malignancy                                    no
cases.0.project.primary_site                                        Breast
cases.0.diagnoses.0.tumor_grade                               not reported
cases.0.project.disease_type                     Breast Invasive Carcinoma
cases.0.diagnoses.0.progression_or_recurrence                 not reported
cases.0.diagnoses.0.tumor_stage                                 stage iiib
cases.0.submitter_id                                          TCGA-AN-A0AL
cancer.type                                                           BRCA
Subtype_mRNA                                                         Basal
Subtype_Selected         